In [4]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
#matplotlib inline
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

In [2]:
import sys
!pip install -Iv https://pypi.python.org/packages/7b/98/7445165b69d4e95403372c07845ad1756af509ac85fc33c5b88e1c3f90c9/xlrd-0.9.0.tar.gz#md5=61102459833cc31d6b05404325fa45a8

  Looking up "https://pypi.python.org/packages/7b/98/7445165b69d4e95403372c07845ad1756af509ac85fc33c5b88e1c3f90c9/xlrd-0.9.0.tar.gz" in the cache
  Current age based on date: 68
  Freshness lifetime from max-age: 31557600
  The response is "fresh", returning cached response
  31557600 > 68
  Using cached xlrd-0.9.0.tar.gz
  Running setup.py (path:/tmp/pip-ta3y2ecq-build/setup.py) egg_info for package from https://pypi.python.org/packages/7b/98/7445165b69d4e95403372c07845ad1756af509ac85fc33c5b88e1c3f90c9/xlrd-0.9.0.tar.gz#md5=61102459833cc31d6b05404325fa45a8
    Running command python setup.py egg_info
    running egg_info
    creating pip-egg-info/xlrd.egg-info
    writing dependency_links to pip-egg-info/xlrd.egg-info/dependency_links.txt
    writing top-level names to pip-egg-info/xlrd.egg-info/top_level.txt
    writing pip-egg-info/xlrd.egg-info/PKG-INFO
    writing manifest file 'pip-egg-info/xlrd.egg-info/SOURCES.txt'
    reading manifest file 'pip-egg-info/xlrd.egg-info/SOURCES.t

In [3]:

# coding: utf-8

# In[1]:

import pandas as pd
import os 
import cv2
import numpy as np


# In[2]:

demographics = pd.read_excel('/src/Data/ANIML/oasis_cross-sectional.xls', sheetname=3) # load data
df = demographics.dropna(how='any') # remove NaN values
df_columns = list(demographics.columns)
X_columns = np.delete(df_columns, [6,7], None) # X matrix won't have MMSE or CDR scores
Xdf = df.reindex(columns=X_columns)
X = Xdf.values # creating X


# In[3]:

# Define function to get list of pngs based on slice number
pngs_path='/src/Data/ANIML/OASIS_pngs'
def getaxialPNG(path):
    l = []
    axialslice90_files = []
    axialslice91_files = []
    axialslice92_files = []
    axialslice93_files = []
    axialslice94_files = []
    axialslice95_files = []
    axialslice96_files = []
    axialslice97_files = []
    axialslice98_files = []
    axialslice99_files = []
    
    for root, directories, filenames in os.walk(path):
        
        for filename in filenames:
            if ".90." in filename: 
                axialslice90_files.append(os.path.join(root, filename))
            if ".91." in filename: 
                axialslice91_files.append(os.path.join(root, filename))
            if ".92." in filename: 
                axialslice92_files.append(os.path.join(root, filename))
            if ".93." in filename: 
                axialslice93_files.append(os.path.join(root, filename))
            if ".94." in filename: 
                axialslice94_files.append(os.path.join(root, filename))
            if ".95." in filename: 
                axialslice95_files.append(os.path.join(root, filename))
            if ".96." in filename: 
                axialslice96_files.append(os.path.join(root, filename))
            if ".97." in filename: 
                axialslice97_files.append(os.path.join(root, filename))
            if ".98." in filename: 
                axialslice98_files.append(os.path.join(root, filename))
            if ".99." in filename: 
                axialslice99_files.append(os.path.join(root, filename))

    l = list(zip(axialslice90_files, axialslice91_files, axialslice92_files, axialslice93_files, axialslice94_files, axialslice95_files, axialslice96_files, axialslice97_files, axialslice98_files, axialslice99_files))

    return ((np.asarray(l)))

axial_files0 = getaxialPNG(pngs_path)
axial_X_files = np.take(axial_files0, indices=df.index.values, axis=0) # keeps the images with the same index as X matrix
axial90loc, axial91loc, axial92loc, axial93loc, axial94loc, axial95loc, axial96loc, axial97loc, axial98loc, axial99loc = zip(*axial_X_files)


# In[4]:

X_id, X_sex, X_handedness, X_age, X_education, X_SES, X_eTIV, X_nWBV, X_ASF = zip(*X) # unzips big X matrix

def sex_translator(X_sex):
    X_sex_binary = []
    X_sex_encoded = []
    for x in X_sex:
        if x == 'M':
            X_sex_binary.append(1)
            X_sex_encoded.append([0,1])
        else:
            X_sex_binary.append(-1)
            X_sex_encoded.append([1,0])
    
    return(zip(X_sex_binary, X_sex_encoded)) # gives us binary and one-hot encoded for sex
           
def hand_translator(X_handedness):
    X_hand_binary = []
    X_hand_encoded = []
    for x in X_handedness:
        if x == 'R':
            X_hand_binary.append(1)
            X_hand_encoded.append([0,1])
        else:
            X_hand_binary.append(-1)
            X_hand_encoded.append([1,0])
    
    return(zip(X_hand_binary, X_hand_encoded)) # same as above but for handedness

# turns out all the patients are right-handed
# maybe we should all just be left-handed so we don't suffer from Alzheimer's
# /frequentist sarcasm

X_sex_binary, X_sex_encoded = zip(*sex_translator(X_sex)) # unzipping to get our function outputs
X_hand_binary, X_hand_encoded = zip(*hand_translator(X_handedness))


# In[5]:

def prepPNGimgs(array_of_image_paths):
    l = []
    for img_file in array_of_image_paths: # for each file in the list of images...
        img = cv2.imread("{}".format(img_file)) # read the image...
        img = cv2.resize(np.array(img), (224,224)) # resize it to 224 by 224 QUICK WHAT'S 224 SQUARED??
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # makes it grayscale
        flatten = gray.flatten() # spaghettifies it into 1 by 50176 (which is 224 squared)
        l.append(flatten)
    
    return(np.asarray(l))


# In[6]:

Y_CDR_columns = [column_name for column_name in df_columns if column_name == 'CDR']
Y_CDR_df = df.reindex(columns=Y_CDR_columns)
Y_CDR = Y_CDR_df.values

Y_MMSE_columns = [column_name for column_name in df_columns if column_name == 'MMSE']
Y_MMSE_df = df.reindex(columns=Y_MMSE_columns)
Y_MMSE = Y_MMSE_df.values

CDR_threshold_0 = 0 # threshold values by CDR scale
CDR_threshold_0point5 = 0.5
CDR_threshold_1 = 1

MMSE_threshold_24 = 24 # threshold values by MMSE scale
MMSE_threshold_18 = 18

def CDR_probable_AD_thresholder(Y_CDR, threshold_value):
    Y_CDR_binary = []
    Y_CDR_encoded = []
    for y in Y_CDR:
        if y > threshold_value:
            Y_CDR_binary.append(1)
            Y_CDR_encoded.append([0,1])
        else:
            Y_CDR_binary.append(-1)
            Y_CDR_encoded.append([1,0])

    return((zip(Y_CDR_binary, Y_CDR_encoded)))

def MMSE_probable_Dementia_thresholder(Y_MMSE, threshold_value):
    Y_MMSE_binary = []
    Y_MMSE_encoded = []
    for y in Y_MMSE:
        if y < threshold_value:
            Y_MMSE_binary.append(1)
            Y_MMSE_encoded.append([0,1])
        else:
            Y_MMSE_binary.append(-1)
            Y_MMSE_encoded.append([1,0])
        
    return(zip(Y_MMSE_binary, Y_MMSE_encoded))

Y_CDR_binary, Y_CDR_encoded = zip(*CDR_probable_AD_thresholder(Y_CDR, CDR_threshold_0))
Y_MMSE_binary, Y_MMSE_encoded = zip(*MMSE_probable_Dementia_thresholder(Y_MMSE, MMSE_threshold_24))


# In[7]:

# turning everything into numpy arrays because why not

df_index = np.asarray(df.index.values)
X_id = np.asarray(X_id)
X_sex = np.asarray(X_sex)
X_sex_binary = np.asarray(X_sex_binary)
X_sex_encoded = np.asarray(X_sex_encoded)
X_handedness = np.asarray(X_handedness)
X_hand_binary = np.asarray(X_hand_binary)
X_hand_encoded = np.asarray(X_hand_encoded)
X_age = np.asarray(X_age) 
X_education = np.asarray(X_education) 
X_SES = np.asarray(X_SES)
X_eTIV = np.asarray(X_eTIV)
X_nWBV = np.asarray(X_nWBV)
X_ASF = np.asarray(X_ASF)
axial90loc = np.asarray(axial90loc) 
axial91loc = np.asarray(axial91loc)
axial92loc = np.asarray(axial92loc) 
axial93loc = np.asarray(axial93loc)
axial94loc = np.asarray(axial94loc)
axial95loc = np.asarray(axial95loc) 
axial96loc = np.asarray(axial96loc) 
axial97loc = np.asarray(axial97loc) 
axial98loc = np.asarray(axial98loc) 
axial99loc = np.asarray(axial99loc)
axial90_spaghetti = prepPNGimgs(axial90loc)
axial91_spaghetti = prepPNGimgs(axial91loc)
axial92_spaghetti = prepPNGimgs(axial92loc)
axial93_spaghetti = prepPNGimgs(axial93loc)
axial94_spaghetti = prepPNGimgs(axial94loc)
axial95_spaghetti = prepPNGimgs(axial95loc)
axial96_spaghetti = prepPNGimgs(axial96loc)
axial97_spaghetti = prepPNGimgs(axial97loc)
axial98_spaghetti = prepPNGimgs(axial98loc)
axial99_spaghetti = prepPNGimgs(axial99loc)
Y_CDR = np.squeeze(np.asarray(Y_CDR))
Y_CDR_binary = np.asarray(Y_CDR_binary)
Y_CDR_encoded = np.asarray(Y_CDR_encoded)
Y_MMSE = np.squeeze(np.asarray(Y_MMSE))
Y_MMSE_binary = np.asarray(Y_MMSE_binary)
Y_MMSE_encoded = np.asarray(Y_MMSE_encoded)


# In[9]:

FinalX = np.vstack((X_id, X_sex, X_sex_binary, X_handedness, X_hand_binary, X_age, X_education, X_SES, X_eTIV, X_nWBV, X_ASF))
FinalX = FinalX.T

FinalY = np.vstack((Y_CDR, Y_CDR_binary, Y_MMSE, Y_MMSE_binary))
FinalY = FinalY.T

/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [24]:
axial_X_files
axial_files0.shape
# np.savetxt("OASIS_subjs.csv", axial90loc, delimiter=",", fmt='<U61')
dt = pd.DataFrame(axial90loc)
dt.to_csv("OASIS_subjs.csv", index=False)

In [4]:
XDATAFRAME = pd.DataFrame((FinalX), index=df_index, columns=['X_id', 'X_sex', 'X_sex_binary', 'X_handedness', 'X_hand_binary', 'X_age', 'X_education', 'X_SES', 'X_eTIV', 'X_nWBV', 'X_ASF'])

YDATAFRAME = pd.DataFrame((FinalY), index=df_index, columns=['Y_CDR', 'Y_CDR_binary', 'Y_MMSE', 'Y_MMSE_binary'])

classifiers = pd.DataFrame((Y_CDR_encoded), index=df_index, columns=['NC','AD'])


In [5]:
axial90s = pd.concat([pd.DataFrame((axial90_spaghetti), index=df_index), classifiers], axis=1)
axial91s = pd.concat([pd.DataFrame((axial91_spaghetti), index=df_index), classifiers], axis=1)
axial92s = pd.concat([pd.DataFrame((axial92_spaghetti), index=df_index), classifiers], axis=1)
axial93s = pd.concat([pd.DataFrame((axial93_spaghetti), index=df_index), classifiers], axis=1)
axial94s = pd.concat([pd.DataFrame((axial94_spaghetti), index=df_index), classifiers], axis=1)
axial95s = pd.concat([pd.DataFrame((axial95_spaghetti), index=df_index), classifiers], axis=1)
axial96s = pd.concat([pd.DataFrame((axial96_spaghetti), index=df_index), classifiers], axis=1)
axial97s = pd.concat([pd.DataFrame((axial97_spaghetti), index=df_index), classifiers], axis=1)
axial98s = pd.concat([pd.DataFrame((axial98_spaghetti), index=df_index), classifiers], axis=1)
axial99s = pd.concat([pd.DataFrame((axial99_spaghetti), index=df_index), classifiers], axis=1)

# axial91s = pd.DataFrame((axial91_spaghetti), index=df_index)
# axial92s = pd.DataFrame((axial92_spaghetti), index=df_index)
# axial93s = pd.DataFrame((axial93_spaghetti), index=df_index)
# axial94s = pd.DataFrame((axial94_spaghetti), index=df_index)
# axial95s = pd.DataFrame((axial95_spaghetti), index=df_index)
# axial96s = pd.DataFrame((axial96_spaghetti), index=df_index)
# axial97s = pd.DataFrame((axial97_spaghetti), index=df_index)
# axial98s = pd.DataFrame((axial98_spaghetti), index=df_index)
# axial99s = pd.DataFrame((axial99_spaghetti), index=df_index)

# divide subject groups into training and test


# allSlices = pd.concat([axial90s, axial91s, axial93s, axial94s, axial95s, axial96s, axial97s, axial98s, axial99s],axis=0)

# #shuffle rows
# allSlicesShuff = allSlices.sample(frac=1, random_state=1).reset_index(drop=True)

# X_train0 = allSlicesShuff.drop(['NC', 'AD'], axis=1)[0:1000].as_matrix()
# y_train0 = allSlicesShuff['AD'][0:1000].as_matrix()

# X_test0 = allSlicesShuff.drop(['NC', 'AD'], axis=1)[1000:].as_matrix()
# y_test0 = allSlicesShuff['AD'][1000:].as_matrix()

In [11]:
axial90s.shape

(216, 50178)

In [6]:
import random
# axial90s.shape
ind= range(0,216)
random.seed(1)
train_ind = random.sample(range(0,216),200)

axials_train = {}
axials_train[1] = axial90s.take(train_ind)
axials_train[2] = axial91s.take(train_ind)
axials_train[3] = axial92s.take(train_ind)
axials_train[4] = axial93s.take(train_ind)
axials_train[5] = axial94s.take(train_ind)
axials_train[6] = axial95s.take(train_ind)
axials_train[7] = axial96s.take(train_ind)
axials_train[8] = axial97s.take(train_ind)
axials_train[9] = axial98s.take(train_ind)
axials_train[10] = axial99s.take(train_ind)


test_ind = list(set(ind) - set(train_ind))

axials_test={}
axials_test[1] = axial90s.take(test_ind)
axials_test[2] = axial91s.take(test_ind)
axials_test[3] = axial92s.take(test_ind)
axials_test[4] = axial93s.take(test_ind)
axials_test[5] = axial94s.take(test_ind)
axials_test[6] = axial95s.take(test_ind)
axials_test[7] = axial96s.take(test_ind)
axials_test[8] = axial97s.take(test_ind)
axials_test[9] = axial98s.take(test_ind)
axials_test[10] = axial99s.take(test_ind)


In [7]:
trainDF = pd.concat(axials_train)
testDF = pd.concat(axials_test)

X_train0 = trainDF.drop(['NC', 'AD'], axis=1).as_matrix()
y_train0 = trainDF['AD'].as_matrix()

X_test0 = testDF.drop(['NC', 'AD'], axis=1).as_matrix()
y_test0 = testDF['AD'].as_matrix()

In [50]:
len(X_train0)
y_train0.shape

(2000,)

In [8]:
# plt.imshow(X_train0[1].reshape(224,224), cmap='gray')
def cannyEdge(imgArray):
    edgesList = []
    for i in range(0, imgArray.shape[0]):
        img = imgArray[i].reshape(224,224)

        nz = np.nonzero(img)

        lower = np.mean(nz)*0.25
        upper = np.mean(nz)*0.75

        edges= cv2.Canny(img, lower, upper)
        flatEdge = edges.flatten()
        edgesList.append(flatEdge)

    return(np.asarray(edgesList))

In [9]:
X_train_edges = cannyEdge(X_train0)
X_test_edges = cannyEdge(X_test0)

In [10]:
dataPath= '/src/Data/ANIML/'
np.save('/src/Data/ANIML/X_train.npy', X_train0)
np.save(dataPath+'y_train.npy', y_train0)
np.save(dataPath+'X_test.npy', X_test0)
np.save(dataPath+'y_test.npy', y_test0)
np.save(dataPath+'AX90.npy', axial90s)
np.save(dataPath+'AX91.npy', axial91s)
np.save(dataPath+'AX92.npy', axial92s)
np.save(dataPath+'AX93.npy', axial93s)
np.save(dataPath+'AX94.npy', axial94s)
np.save(dataPath+'AX95.npy', axial95s)
np.save(dataPath+'AX96.npy', axial96s)
np.save(dataPath+'AX97.npy', axial97s)
np.save(dataPath+'AX98.npy', axial98s)
np.save(dataPath+'AX99.npy', axial99s)
np.save(dataPath+'X_train_edges.npy', X_train_edges)
np.save(dataPath+'X_test_edges.npy', X_test_edges)

In [10]:
import numpy as np
import keras
from keras import optimizers
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import MaxPooling2D,Input, Convolution2D, Dense, Dropout, Flatten, Concatenate, Merge, merge
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from load_data import load
from metadata import prepare_metadata
###### get IDs and labels ######
# fn = '/Users/newhaven510/Desktop/UCLA_Spring_18/EE210A/OASIS3_noNA_ybin.csv'
fn = '/Users/newhaven510/Downloads/new_Y_Data.csv'

dirimg = '/Users/newhaven510/Desktop/UCLA_Spring_18/EE210A/Original_cropped/'
dirjac = '/Users/newhaven510/Desktop/UCLA_Spring_18/EE210A/Jac_cropped/'
data = load(fn, dirimg, dirjac)
Y_train0 = data.y_train0
Y_test0 = data.y_test0
###### smooth and normalize ######
X_train0 = data.train_data_matrix[:,61,:,:]
X_test0 = data.test_data_matrix[:,61,:,:]


In [11]:
###### set data #######
X_train, Y_train = X_train0, Y_train0
X_train= X_train.astype('float32')
# for i in range(0,X_train.shape[0]):
#     X_train[i,:,:] /= X_train[i,:,:].max()
X_train/=255

X_test, Y_test = X_test0, Y_test0
X_test = X_test.astype('float32')
# for i in range(0,X_test.shape[0]):
#     X_test[i,:,:] /= X_test[i,:,:].max()
X_test/=255
X_train = np.expand_dims(X_train, axis =3)
X_test = np.expand_dims(X_test, axis =3)

In [30]:
####Model Functional
number_of_classes = 2
Y_train = np_utils.to_categorical(Y_train, number_of_classes)
Y_train = Y_train.astype('float32')
Y_test = np_utils.to_categorical(Y_test, number_of_classes)
Y_test = Y_test.astype('float32')
from sklearn.utils import shuffle

X_train,Y_train = shuffle(X_train,Y_train)


In [12]:
np.sum(Y_train)

58.0

In [17]:
keras.__version__

'2.1.6'

In [34]:
from numpy.random import seed

class LeNetMnistClassifier():
    @staticmethod
    def toDigit(hot_encode):
        return np.argmax(hot_encode)
        
    def __init__(self, **kwargs):
        if kwargs.get("model_path", None):
            from keras.models import load_model
            self.model = load_model(kwargs['model_path'])
        else:
            self.activation_function = kwargs.pop('activation_function', 'relu')
            self.batch_size = kwargs.pop('batch_size', 15)
            self.epochs = kwargs.pop('epochs', 5)
            self.kernal_size = kwargs.pop('kernal_size', (3, 3))
            self.loss_function = kwargs.pop('loss_function', 'categorical_crossentropy')
            self.optimizer = kwargs.pop('optimizer', 'sgd')
            self.pool_size = kwargs.pop('pool_size', (2, 2))
            self.model = self._model(X_train)#, X_train2)
            
    
    def _model(self, X_train):#, X_train2):
        seed(1)
        model = Sequential()#, X_train2])
#         Model
        model.add(Conv2D(32, kernel_size=self.kernal_size, padding='same',
                 activation=self.activation_function,
                 input_shape=(64,64,1)))
        # One additional convolutional layer (32 channels)
        model.add(Conv2D(32, kernel_size=self.kernal_size, padding='same',
                 activation=self.activation_function))
        model.add(Conv2D(32, kernel_size=self.kernal_size, padding='same',
                 activation=self.activation_function))
        model.add(MaxPooling2D(pool_size=self.pool_size))
        model.add(Conv2D(64, self.kernal_size, padding='same', activation=self.activation_function))
        # One additional convolutional layer (64 channels)
        model.add(Conv2D(64, self.kernal_size, padding='same', activation=self.activation_function))
        model.add(Conv2D(64, self.kernal_size, padding='same', activation=self.activation_function))
        model.add(MaxPooling2D(pool_size=self.pool_size))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(512, activation=self.activation_function))
        model.add(Dropout(0.25))
        model.add(Dense(2, activation='softmax'))
        
        return model
    
    def predict(self, digits=()):
        return self.model.predict(digits)
    
    def evaluate(self, X_test=None, y_test=None):
        X_test = X_test.reshape(X_test.shape[0], 224, 224, 1)
        X_test = X_test.astype('float32')
        X_test/=255        
        number_of_classes = 2
        y_test = np_utils.to_categorical(y_test, number_of_classes)
        return self.model.evaluate(X_test, y_test)
    
    def preprocess_and_train(self, X_train=None, y_train=None, X_test=None,y_test=None):
        self._train(X_train,  y_train, X_test, y_test)
        
    def _preprocess(self, X_train, X_train2, y_train, X_test, X_test2,y_test):
        X_train = X_train.reshape(X_train.shape[0], 224, 224, 1)
        X_test = X_test.reshape(X_test.shape[0], 224, 224, 1)

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        
        X_train/=255
        X_test/=255
        
        X_train2 = X_train2.reshape(X_train2.shape[0], 224, 224, 1)
        X_test2 = X_test2.reshape(X_test2.shape[0], 224, 224, 1)

        X_train2 = X_train2.astype('float32')
        X_test2 = X_test2.astype('float32')
        
        X_train2/=255
        X_test2/=255
        
        
        number_of_classes = 2
        y_train = np_utils.to_categorical(y_train, number_of_classes)
        y_test = np_utils.to_categorical(y_test, number_of_classes)
        
        return X_train, X_train2, y_train, X_test, X_test2, y_test
    
#     def _train(self, X_train, X_train2, y_train, X_test, X_test2, y_test):
    def _train(self, X_train, y_train, X_test, y_test):
        seed(1)
        self.model.compile(loss=self.loss_function,
              optimizer=self.optimizer,
              metrics=['accuracy'])
        
        fit_output = self.model.fit([X_train],
                        y_train,
                        batch_size=self.batch_size,
                        epochs=self.epochs,
                        verbose=1,
                        validation_data=([X_test], y_test))
        self._history = fit_output.history
        
        import time
        timestr = time.strftime("%Y%m%d-%H%M%S")
        self.model.save("oasis_test_{0}.h5".format(timestr)) 
    
    @property
    def history(self):
        return self._history

In [33]:
classifier = LeNetMnistClassifier(X_train=X_train,epochs=30, optimizer=keras.optimizers.Adadelta())
classifier.preprocess_and_train(X_train=X_train,y_train=Y_train, X_test=X_test, y_test=Y_test)

Train on 166 samples, validate on 50 samples
Epoch 1/30
166/166 [==============================] - 9s 54ms/step - loss: 1.0047 - acc: 0.5482 - val_loss: 1.1488 - val_acc: 0.5000
Epoch 2/30
166/166 [==============================] - 9s 52ms/step - loss: 0.6855 - acc: 0.6566 - val_loss: 0.7899 - val_acc: 0.5000
Epoch 3/30
166/166 [==============================] - 8s 49ms/step - loss: 0.6544 - acc: 0.6506 - val_loss: 0.7887 - val_acc: 0.5000
Epoch 4/30
166/166 [==============================] - 9s 52ms/step - loss: 0.6497 - acc: 0.6506 - val_loss: 0.6976 - val_acc: 0.5000
Epoch 5/30
166/166 [==============================] - 8s 48ms/step - loss: 0.6578 - acc: 0.6506 - val_loss: 0.6947 - val_acc: 0.5000
Epoch 6/30
166/166 [==============================] - 8s 49ms/step - loss: 0.7082 - acc: 0.6506 - val_loss: 0.6984 - val_acc: 0.5000
Epoch 7/30
166/166 [==============================] - 8s 50ms/step - loss: 0.6576 - acc: 0.6506 - val_loss: 1.0145 - val_acc: 0.5000
Epoch 8/30
166/166 [====

In [5]:
# from keras.models import load_model
# model_0p6 = load_model('0p6_lenet.h5')
# model_0p6.layers.pop()
# # model_0p6.summary()
# model_0p6.build()
# model_0p6.save('0p6_lenet_pop.h5')

In [6]:
intermediate_layer = Model(inputs=model_0p6.input, outputs=model_0p6.get_layer(index=11).output)
# print(intermediate_layer)
intermediate_output = intermediate_layer.predict(X_train)

In [7]:
intermediate_output_test = intermediate_layer.predict(X_test)

In [8]:
print(intermediate_output.shape)

(166, 512)


In [14]:
np.sum(Y_train)

58.0

In [15]:
import pandas as pd
from sklearn import svm

demog = pd.read_csv(fn)

# print(demog)

# demogDF = pd.concat((pd.DataFrame(intermediate_output), demog[['X_sex_binary', 'X_age', 'X_education',
#                                                  'X_SES', 'X_eTIV', 'X_nWBV', 'X_ASF', 
#                                                  'Y_MMSE']]), axis=1)
subdemog = demog[demog['Subject'].isin(data.y_train0_id)].reset_index()
demogDF = pd.concat((pd.DataFrame(intermediate_output), subdemog[['X_sex_binary', 'X_age','X_eTIV', 'X_nWBV', 'X_ASF', 
                                                 'Y_MMSE']]), axis=1)
metaTrain = np.asmatrix(subdemog[['X_sex_binary', 'X_age','X_eTIV', 'X_nWBV', 'X_ASF', 
                                                 'Y_MMSE']])
demogDFmat = np.asmatrix(demogDF)


In [119]:
print(clf)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [128]:
print(metaTrain[1,:])

[[-1.00000e+00  7.00000e+01  1.28285e+03  7.91000e-01  1.36805e+00
   2.90000e+01]]


In [ ]:
from sklearn.preprocessing import normalize
best = 0
best_c = -10
metaTrainNorm = normalize(metaTrain)#normalize(demogDFmat)
metaTestNorm = normalize(metaTest)#normalize(demogDFmattest)

for i in np.logspace(4,15,50):
    clf = svm.SVC(C=i)
    clf.fit(metaTrainNorm, np.asarray(Y_train0))
    if(clf.score(metaTestNorm,Y_test)>best):
        best = clf.score(metaTestNorm,Y_test)
        best_c = i
print(best,np.log10(best_c))

In [124]:
print(best,best_c)
clf.predict(demogDFmattest)

0.5 1e-05


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
subdemogtest = demog[demog['Subject'].isin(data.y_test0_id)].reset_index()
demogDFtest = pd.concat((pd.DataFrame(intermediate_output_test), subdemogtest[['X_sex_binary', 'X_age','X_eTIV', 'X_nWBV', 'X_ASF', 
                                                 'Y_MMSE']]), axis=1)
metaTest = np.asmatrix(subdemogtest[['X_sex_binary', 'X_age','X_eTIV', 'X_nWBV', 'X_ASF', 
                                                 'Y_MMSE']])
demogDFmattest = np.asmatrix(demogDFtest)
clf.predict(metaTest)
# print(subdemogtest)

NameError: name 'clf' is not defined

In [86]:

clf.score(metaTest,Y_test)

0.48

In [35]:
classifier = LeNetMnistClassifier(epochs=30, optimizer=keras.optimizers.Adadelta())
classifier.preprocess_and_train(X_train=X_train, y_train=Y_train, X_test=X_test, y_test=Y_test)

Train on 166 samples, validate on 50 samples
Epoch 1/30
166/166 [==============================] - 10s 58ms/step - loss: 0.9702 - acc: 0.5241 - val_loss: 0.6998 - val_acc: 0.5000
Epoch 2/30
166/166 [==============================] - 10s 58ms/step - loss: 0.6923 - acc: 0.6506 - val_loss: 0.6951 - val_acc: 0.5000
Epoch 3/30
166/166 [==============================] - 9s 54ms/step - loss: 0.6686 - acc: 0.6506 - val_loss: 0.6938 - val_acc: 0.5000
Epoch 4/30
166/166 [==============================] - 9s 51ms/step - loss: 0.6688 - acc: 0.6506 - val_loss: 0.6939 - val_acc: 0.5000
Epoch 5/30
166/166 [==============================] - 8s 48ms/step - loss: 0.6594 - acc: 0.6446 - val_loss: 1.0612 - val_acc: 0.5000
Epoch 6/30
166/166 [==============================] - 8s 51ms/step - loss: 0.6788 - acc: 0.6506 - val_loss: 0.9073 - val_acc: 0.5000
Epoch 7/30
166/166 [==============================] - 8s 49ms/step - loss: 0.6418 - acc: 0.6506 - val_loss: 0.6924 - val_acc: 0.5000
Epoch 8/30
166/166 [==

In [2]:
import tensorflow as tf
tf.Session(config=tf.ConfigProto(log_device_placement=True))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

print(tf.__version__)

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13154147634325830913
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 104136704
locality {
  bus_id: 1
}
incarnation: 18100578659675994095
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0"
]
1.2.1


In [ ]:
## prepare data set
